In [5]:
%reset

In [1]:
import pandas as pd
import numpy as np
import random

#flights = pd.read_csv('../../../large_files/flights_w_weather_full.csv')

In [2]:
flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name', 'city_name_dep', 'conditions_dep',
       'is_sunny_dep', 'is_cloudy_dep', 'is_rain_dep', 'is_snow_dep', '_merge',
       'city_name_arr', 'conditions_arr', 'is_sunny_arr', 'is_cloudy_arr',
       'is_rain_arr', 'is_snow_arr', 'merge_arrivals'],
      dtype='ob

In [4]:
#flights = copy.deepcopy(flights_)#.sample(10**6)
cols = ['fl_date','op_unique_carrier','origin_airport_id','dest_airport_id','dep_time','dep_delay','taxi_out','cancelled','cancellation_code','carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay','conditions_dep','is_sunny_dep','is_cloudy_dep','is_rain_dep','is_snow_dep']
flights = flights[cols]

The target variables are CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

In [5]:
def binar(arr,eps=2**-16):
    c,w,n,s,l = [i for i in arr]
    max_delay = max([c,w,n,s,l,1])
 
    k = [k for k,j in enumerate([c,w,n,s,l]) if j == max_delay]
    
    if not k:
       k = [random.randint(0,4)]
    rand_gen = random.choice(k)
    
    return int(c+int(rand_gen == 0)*eps > max_delay),int(w+int(rand_gen == 1)*eps > max_delay),int(n+int(rand_gen == 2)*eps > max_delay),int(s+int(rand_gen == 3)*eps > max_delay),int(l+int(rand_gen == 4)*eps > max_delay)

indys = flights.loc[~flights.carrier_delay.isna()].index
flights.loc[indys,['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']]= flights.loc[indys,['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].apply(lambda x: binar(x),raw=True,axis=1,result_type='broadcast')

In [11]:
c_delay, w_delay, n_delay, s_delay, l_delay = flights.loc[indys,['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].apply(lambda x: x.sum(),raw=True)/len(flights.loc[indys])*100
print(c_delay, w_delay, n_delay, s_delay, l_delay)

26.525123036141117 3.348065321489249 30.503025718608168 0.17179195392824953 39.45192749974077


In [6]:
flights['delay_code'] = np.nan
flights.loc[indys,'delay_code'] = flights.loc[indys,['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].apply(lambda x: x.tolist().index(max(x)),axis=1,raw=True)

In [13]:
flights.to_csv('../../../large_files/flights_transformed.csv')

In [7]:
flights.sort_values(by=['op_unique_carrier','origin_airport_id','fl_date'],inplace=True)

In [8]:
flights['month'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.month)
flights['year'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.year)

In [9]:
flights.columns

Index(['fl_date', 'op_unique_carrier', 'origin_airport_id', 'dest_airport_id',
       'dep_time', 'dep_delay', 'taxi_out', 'cancelled', 'cancellation_code',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'conditions_dep', 'is_sunny_dep',
       'is_cloudy_dep', 'is_rain_dep', 'is_snow_dep', 'delay_code', 'month',
       'year'],
      dtype='object')

In [10]:
carrier_ids = flights.op_unique_carrier.unique().tolist()
flights['rolling_dep_delay_carr-airport'] = 0
for i in carrier_ids:
    indy = flights[flights['op_unique_carrier']==i].index
    temp = flights.loc[indy]
    airport_ids = temp.origin_airport_id.unique().tolist()
    for j in airport_ids:
        indy2 = temp[temp['origin_airport_id']==j].index
        flights.loc[indy2,'rolling_dep_delay_carr-airport'] = temp.loc[indy2,'dep_delay'].rolling(7,min_periods=1).mean()
flights['average_dep_delay_airport-month'] = 0
airport_ids = flights.origin_airport_id.unique().tolist()
for i in airport_ids:
    indy = flights[flights['origin_airport_id']==i].index
    temp = flights.loc[indy]
    months = temp.month.unique().tolist()
    for month in months:
        indy2 = temp[temp['month']==month].index
        flights.loc[indy2,'average_dep_delay_airport-month'] = flights.loc[indy2,'dep_delay'].mean()

In [11]:
flights.to_csv('../../../large_files/flights_transformed.csv')

In [2]:
flights = pd.read_csv('../../../large_files/flights_transformed.csv',index_col=0)
passengers = pd.read_csv('../../../large_files/passengers.csv')

In [3]:
col_p = ['departures_performed', 'passengers', 'unique_carrier', 'month', 'year', 'origin_airport_id', 'dest_airport_id']
passengers = passengers[col_p]
passengers = passengers[passengers.passengers>0]
passengers = passengers[passengers.year>2018]

In [4]:
op_unique_carrier = flights.op_unique_carrier.unique().tolist()
passengers = passengers[passengers.unique_carrier.isin(op_unique_carrier)]

In [5]:
passengers['passengers'] /= passengers.departures_performed
passengers

,departures_performed,passengers,unique_carrier,month,year,origin_airport_id,dest_airport_id
361,4.0,43.000000,EM,5,2019,12173,13034
362,4.0,33.500000,EM,5,2019,13034,12173
363,1.0,66.000000,G7,5,2019,10140,11292
364,1.0,66.000000,G7,5,2019,10397,10140
365,1.0,15.000000,G7,5,2019,10397,11109
...,...,...,...,...,...,...,...
2350446,594.0,5.508418,9K,9,2019,10721,10154
2350448,596.0,5.968121,9K,7,2019,10721,10154
2350449,596.0,5.369128,9K,6,2019,10154,10721
2350452,617.0,5.931929,9K,9,2019,10154,10721


In [10]:
#flights = flights.merge(passengers, how='inner', left_on=['op_unique_carrier','origin_airport_id','dest_airport_id','month','year'],right_on=['unique_carrier','origin_airport_id', 'dest_airport_id','month','year'])
flights.drop_duplicates(subset=['fl_date','op_unique_carrier','origin_airport_id', 'dest_airport_id','dep_time'],inplace=True)

In [2]:
#flights.to_csv('../../../large_files/flights_transformed_w_pax.csv')
flights = pd.read_csv('../../../large_files/flights_transformed_w_pax.csv',index_col=0)

In [3]:
flights.columns

Index(['fl_date', 'op_unique_carrier', 'origin_airport_id', 'dest_airport_id',
       'dep_time', 'dep_delay', 'taxi_out', 'cancelled', 'cancellation_code',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'conditions_dep', 'is_sunny_dep',
       'is_cloudy_dep', 'is_rain_dep', 'is_snow_dep', 'delay_code', 'month',
       'year', 'rolling_dep_delay_carr-airport',
       'average_dep_delay_airport-month', 'departures_performed', 'passengers',
       'unique_carrier'],
      dtype='object')

In [4]:
flights['day'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.day)
flights['day_of_week'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d').apply(lambda x: x.dayofweek)
flights['hour'] = round(flights.dep_time%100/60+(flights.dep_time-flights.dep_time%100)/100)

In [5]:
pax_hourly = flights.groupby(['origin_airport_id','year','month','day','hour'])['passengers'].sum().reset_index(level=['origin_airport_id','year','month','day','hour'])
traffic_hourly = flights.groupby(['origin_airport_id','year','month','day','hour']).size().reset_index(level=['origin_airport_id','year','month','day','hour'])
traffic_hourly['pax-hourly-traffic'] = pax_hourly.passengers
traffic_hourly

,origin_airport_id,year,month,day,hour,0,pax-hourly-traffic
0,10135,2019,1,1,5.0,1,35.093023
1,10135,2019,1,1,6.0,2,120.734177
2,10135,2019,1,1,8.0,1,46.428571
3,10135,2019,1,1,11.0,2,210.877034
4,10135,2019,1,1,14.0,2,187.482759
...,...,...,...,...,...,...,...
1208413,16869,2019,12,30,17.0,1,46.971831
1208414,16869,2019,12,31,6.0,2,113.771831
1208415,16869,2019,12,31,13.0,1,66.800000
1208416,16869,2019,12,31,15.0,1,46.971831


In [6]:
flights = flights.merge(traffic_hourly,how='inner',on=['origin_airport_id','year','month','day','hour'])
flights.rename(columns={0:'airport_hourly_traffic'})
flights

,fl_date,op_unique_carrier,origin_airport_id,dest_airport_id,dep_time,dep_delay,taxi_out,cancelled,cancellation_code,carrier_delay,...,rolling_dep_delay_carr-airport,average_dep_delay_airport-month,departures_performed,passengers,unique_carrier,day,day_of_week,hour,0,pax-hourly-traffic
0,2019-01-01,9E,10135,10397,557.0,-3.0,23.0,0.0,NaN,NaN,...,38.571429,12.514763,1.0,53.000000,9E,1,1,6.0,2,120.734177
1,2019-01-01,OH,10135,11057,552.0,-9.0,12.0,0.0,NaN,NaN,...,1.571429,12.514763,79.0,67.734177,OH,1,1,6.0,2,120.734177
2,2019-01-02,9E,10135,10397,554.0,-6.0,16.0,0.0,NaN,NaN,...,14.000000,12.514763,1.0,53.000000,9E,2,2,6.0,4,199.309959
3,2019-01-02,OH,10135,11057,601.0,0.0,12.0,0.0,NaN,NaN,...,-5.000000,12.514763,79.0,67.734177,OH,2,2,6.0,4,199.309959
4,2019-01-02,OO,10135,11433,530.0,-5.0,20.0,0.0,NaN,NaN,...,61.000000,12.514763,86.0,35.093023,OO,2,2,6.0,4,199.309959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934145,2019-06-01,ZW,15919,13930,1055.0,-2.0,23.0,0.0,NaN,0.0,...,19.714286,18.857816,34.0,40.764706,ZW,1,5,11.0,1,40.764706
7934146,2019-06-13,ZW,15919,13930,1504.0,-6.0,16.0,0.0,NaN,NaN,...,76.666667,18.857816,34.0,40.764706,ZW,13,3,15.0,1,40.764706
7934147,2019-08-20,ZW,15919,13930,1541.0,303.0,46.0,0.0,NaN,0.0,...,55.142857,12.164542,34.0,36.470588,ZW,20,1,16.0,1,36.470588
7934148,2019-08-27,ZW,15919,13930,806.0,-15.0,12.0,0.0,NaN,NaN,...,20.500000,12.164542,34.0,36.470588,ZW,27,1,8.0,1,36.470588


In [7]:
flights.to_csv('../../../large_files/flights_transformed_w_pax.csv')
flights_filtered = flights[~flights['carrier_delay'].isna()]

In [8]:
flights_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1525492 entries, 6 to 7934147
Data columns (total 32 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   fl_date                          1525492 non-null  object 
 1   op_unique_carrier                1525492 non-null  object 
 2   origin_airport_id                1525492 non-null  int64  
 3   dest_airport_id                  1525492 non-null  int64  
 4   dep_time                         1525492 non-null  float64
 5   dep_delay                        1525492 non-null  float64
 6   taxi_out                         1525492 non-null  float64
 7   cancelled                        1525492 non-null  float64
 8   cancellation_code                0 non-null        object 
 9   carrier_delay                    1525492 non-null  float64
 10  weather_delay                    1525492 non-null  float64
 11  nas_delay                        1525492 non-null 

In [9]:
flights_filtered.drop(columns=['cancellation_code'],inplace=True)
flights_filtered.to_csv('../../../large_files/flights_multiclass_data.csv')